In [33]:
from azureml.core import Workspace, Experiment
from azureml.core import Workspace

ws = Workspace.from_config()
ws.write_config(path='.azureml')

experiment_name = 'udacity_project_solution_real'
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,  
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132620
Azure region: southcentralus
Resource group: aml-quickstarts-132620


In [34]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "optimizeml"

try:
    cpu_cluster = ComputeTarget(workspace=ws,name=cpu_cluster_name)
    print("CPU Cluster exist")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

CPU Cluster exist
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
# from azureml.train.sklearn import SKLearn
# SKLearn?
# from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# HyperDriveConfig?

In [35]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, choice
import os
import shutil


# Specify parameter sampler
ps = RandomParameterSampling( 
    {'--C' : choice(1,2,3,4,5),
     '--max_iter' : choice(100,150,200,250)}
)

# Specify a Policy

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_folder,compute_target=cpu_cluster,entry_script='train.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20)

In [36]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
experiment = Experiment(workspace=ws, name='HyperDriveMLOptimize')
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [41]:
import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '2', '--max_iter', '250']


In [42]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_8a8b65c643bf10b45b90de334cfeaef2c6faa6be05116531d07234558c1955c8_d.txt', 'azureml-logs/65_job_prep-tvmps_8a8b65c643bf10b45b90de334cfeaef2c6faa6be05116531d07234558c1955c8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_8a8b65c643bf10b45b90de334cfeaef2c6faa6be05116531d07234558c1955c8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [43]:
#best_run.register_model?

In [44]:
model = best_run.register_model(model_name='logistic_hyperdrive', model_path='outputs/model.joblib')

In [45]:
# import joblib
# # Get your best run and save the model from that run.
# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

### YOUR CODE HERE ###

Best Run Id:  HD_f31498ad-df25-41fd-8af8-8856ed8389c0_19

 Accuracy: 0.9120890237733941


In [47]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
auto_loc = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ads = TabularDatasetFactory.from_delimited_files(auto_loc, separator=',') ### YOUR CODE HERE ###
### YOUR CODE HERE ###

In [48]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ads)

In [67]:
data = x.join(y)
type(data['y'])

pandas.core.series.Series

In [51]:
from azureml.train.automl.utilities import get_primary_metrics 
get_primary_metrics('classification')

['AUC_weighted',
 'accuracy',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'precision_score_weighted']

In [56]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data,
    label_column_name=y,
    n_cross_validations=5)

In [57]:
from azureml.core.experiment import Experiment


# Choose a name for the experiment and specify the project folder.
experiment_name = 'logistic-automl'
#project_folder = './sample_projects/automl-classification'

experiment = Experiment(ws, experiment_name)

In [58]:
# Submit your automl run
run = experiment.submit(automl_config, show_output=True)
### YOUR CODE HERE ###

ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

In [ ]:
########testing########
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

# Specify parameter sampler # OWN CODE
ps = RandomParameterSampling({
    "--C": uniform(0.0, 1),
    "--max_iter":choice(50, 100, 150, 200, 250)
})

# Specify a Policy ### OWN code 
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory = script_folder, compute_target = optimizeml, vm_size = 'STANDARD_D2_V2', vm_priority = 'lowpriority',
              entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = ps,
                                     policy = policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs = 1,
                                     max_total_runs = 1)